# Setup

In [1]:
from image_compressor.svd_compressor import SVDCompressor
from image_compressor.svd import svd
from image_compressor.compressed import Compressed
from image_compressor.evaluation import mse, psnr, pearson_correlation, cr

In [2]:
# add metrics
def er_score(image, U, singular_values, V, rank):
    original_energy = np.sum(singular_values ** 2)
    current_energy = np.sum(singular_values[:rank] ** 2)
    return current_energy / original_energy

def ssim_score(image, U, singular_values, V, rank, window_size=8):
    decompressed = (U[:, :rank] @ np.diag(singular_values[:rank]) @ V[:rank, :]).astype(np.float32)
    m, n = image.shape
    simmilarity = 0
    for y in range(0, m - window_size, window_size):
        for x in range(0, n - window_size, window_size):
            simmilarity += ssim_window(image[y:y + window_size, x:x + window_size],
                                          decompressed[y:y + window_size, x:x + window_size])
    windows_number = (m//window_size)*(n//window_size)
    simmilarity /= windows_number
    return simmilarity

def ssim_window(X, Y):
    L = 255
    k1 = 0.01
    k2 = 0.03
    c1 = (k1 * L) ** 2
    c2 = (k2 * L) ** 2
    c3 = c2 / 2
    
    mu1, mu2 = np.average(X), np.average(Y)
    luminance = (2 * mu1 * mu2 + c1) / (mu1 ** 2 + mu2 ** 2 + c1)
    std1, std2 = np.std(X), np.std(Y)
    contrast = (2 * std1 * std2 + c2) / (std1 ** 2 + std2 ** 2 + c2)
    covariance = np.matmul((X - mu1).flatten(), (Y - mu2).flatten()) / X.size
    structure = (covariance + c3) / (std1 * std2 + c3)
    return luminance * contrast * structure

In [3]:
# update svd compressor that also take rank in function compress and return image, ssim and er
class SVDCompressor_rank():
    def compress(self, image, ranks):
        image = image.astype(np.float32)
        m, n = image.shape
        transposed = False
        if n > m:
            image = image.T
            transposed = True
        U, S, V = svd(image)
        if n > m:
            U, V = V.T, U.T

        singular_values = np.diag(S)
        indexes = np.argsort(singular_values)[::-1]
        U = U[:, indexes]
        V = V[indexes, :]
        singular_values = singular_values[indexes]
        if transposed:
            image = image.T
            
        if type(ranks) == list:
            return [[Compressed(U[:, :rank], singular_values[:rank], V[:rank, :]), \
                    ssim_score(image, U, singular_values, V, rank), er_score(image, U, singular_values, V, rank)] \
                    for rank in ranks]
        else:            
            return Compressed(U[:, :ranks], singular_values[:ranks], V[:ranks, :]), \
                    ssim_score(image, U, singular_values, V, ranks), er_score(image, U, singular_values, V, ranks)

# Experiments

In [ ]:
import numpy as np
import pandas
import cv2
import matplotlib.pyplot as plt

def compress_image(image, ranks):
    compressor = SVDCompressor_rank()
    compressor_data = compressor.compress(image, ranks)
    return [[compressed.to_image(), compressed.get_effective_rank(), mse(image, compressed), psnr(image, compressed),\
            pearson_correlation(image, compressed), cr(image, compressed), ssim, er] for compressed, ssim, er in compressor_data]

In [ ]:
# set ranks
ranks = [2**i for i in range(9)]

In [ ]:
image = cv2.imread("image/barack_obama.jpg")
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite("image/obama_512.jpg", image)

output = compress_image(image, ranks)
table = pandas.DataFrame([i[1:] for i in output], columns = ["Rank", "MSE", "PSNR", "PR", "CR", "SSIM", "ER"])

for i in range(len(output)):
    cv2.imwrite("image/obama_compressed_{}.jpg".format(output[i][1]), output[i][0])

In [ ]:
# round dloats
table1 = table.copy()
# second item in tuple number of decimal places
columns_to_digits = [("MSE", 1), ("PSNR", 1), ("PR", 4), ("SSIM", 4), ("ER", 4), ("CR", 1)]

for column, digits in columns_to_digits:
    table1[column] = table1[column].apply(lambda x: round(x, digits)) if digits > 0 else \
                     table1[column].apply(lambda x: int(x))
table1.to_csv("table.csv", index= False)
table1

In [ ]:
# plot everything
for column in ["MSE", "PSNR", "PR", "CR", "SSIM", "ER"]:
    plt.plot(table["Rank"], table[column])
    plt.xlabel("Rank")
    plt.ylabel(column)
    plt.xscale('log', basex=2)
    plt.savefig("plots/{}_to_rank.jpg".format(column.lower()))
    plt.show()